In [1]:
import pandas as pd 
import numpy as np


In [2]:
df = pd.read_csv('final_dataset_DDate.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711341 entries, 0 to 711340
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DDate           711341 non-null  object 
 1   AmcName         711341 non-null  object 
 2   YardName        711341 non-null  object 
 3   CommName        711341 non-null  object 
 4   VarityName      711341 non-null  object 
 5   Arrivals        711341 non-null  float64
 6   Minimum         711341 non-null  float64
 7   Maximum         711341 non-null  float64
 8   Model           711341 non-null  float64
 9   Year            711341 non-null  int64  
 10  Month           711341 non-null  int64  
 11  DayOfWeek       711341 non-null  int64  
 12  WeekOfYear      711341 non-null  int64  
 13  IsRabiSeason    711341 non-null  int64  
 14  IsKharifSeason  711341 non-null  int64  
 15  PriceSpread     711341 non-null  float64
 16  Volatility      711341 non-null  float64
 17  LogArrival

In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import json

# Load your DataFrame `df` here
# df = pd.read_csv("your_file.csv")  # or however you load it

# Step 1: Convert DDate to datetime
df['DDate'] = pd.to_datetime(df['DDate'])

# Step 2: Encode categorical variables and save encoders
categorical_cols = ['AmcName', 'YardName', 'CommName', 'VarityName']
encoding_map = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoding_map[col] = dict(zip(le.classes_, le.transform(le.classes_)))

# Save encoding_map to a JSON file
import os
os.makedirs('output', exist_ok=True)
with open('output/encoding_map.json', 'w') as f:
    json.dump(encoding_map, f)

# Step 3: Resample/aggregate by date (if needed)
df_sorted = df.sort_values('DDate')
df_grouped = df_sorted.groupby('DDate')['Model'].mean().asfreq('D')  # Daily average

# Handle missing values (optional)
df_grouped = df_grouped.fillna(method='ffill')

# Step 4: Train/Test Split
train_size = int(len(df_grouped) * 0.8)
train, test = df_grouped.iloc[:train_size], df_grouped.iloc[train_size:]

# Step 5: Fit ARIMA model
model = ARIMA(train, order=(5, 1, 0))
model_fit = model.fit()

# Step 6: Forecast
forecast = model_fit.forecast(steps=len(test))
forecast.index = test.index

# Step 7: Evaluation
mae = mean_absolute_error(test, forecast)
rmse = mean_squared_error(test, forecast, squared=False)
r2 = r2_score(test, forecast)

print(f"\n✅ ARIMA Model Performance:")
print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.2f}")

# Step 8: Save ARIMA model
with open('/arima_model.pkl', 'wb') as f:
    pickle.dump(model_fit, f)

print("\n📁 Files saved:")
print("- /mnt/data/arima_model.pkl")
print("- /mnt/data/encoding_map.json")


TypeError: keys must be str, int, float, bool or None, not int64